In [ ]:
import pandas as pd
from collections import defaultdict
import os
import math
import glob
import json

In [ ]:
import pandas as pd
from collections import defaultdict
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import os
import math
import itertools

In [ ]:
import swifter

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
ALL_SCHOOLS_DATA_DIR = '/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/all_schools_data_orig/csvs/'


In [ ]:
total_number_of_years = 10

In [ ]:
all_files = glob.glob(os.path.join(ALL_SCHOOLS_DATA_DIR, "*.csv"))


In [ ]:
assert(len(all_files) == 4886)

In [ ]:
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df['inv_unique_id'] = df['provider_and_id'].astype(str) + '_' +  df['involve_id'].astype(str) + '_' + df['accident_year'].astype(str)


In [ ]:
df['acc_unique_id'] = df['provider_and_id'].astype(str) + '_' + df['accident_year'].astype(str)


In [ ]:
df['vehicle_or_pedestrian'] = df.apply(lambda x: x['involve_vehicle_type_hebrew'] if x['injured_type_hebrew'] != 'הולך רגל' else  x['injured_type_hebrew'], axis=1)


In [ ]:
df['accident_timestamp'] = pd.to_datetime(df['accident_timestamp'])

### Extract Yom Kippur

In [ ]:
from datetime import date
import holidays
yom_kippuer_dates = []
for date, name in sorted(holidays.IL(years=range(2013,2023)).items()):
    if "Yom Kippur" in name:
        yom_kippuer_dates.append(date)

In [ ]:
assert(len(yom_kippuer_dates) == total_number_of_years * 2)

In [ ]:
df = df.loc[~(df.accident_timestamp.dt.date.isin(yom_kippuer_dates))]

In [ ]:
assert(not(df.accident_timestamp.dt.date.isin(yom_kippuer_dates).sum()))

In [ ]:
months_dict = {1: 'ינואר', 
               2: 'פברואר',
               3: 'מרס',
               4: 'אפריל',
               5: 'מאי',
               6: 'יוני',
               7: 'יולי',
               8: 'אוגוסט',
               9: 'ספטמבר',
               10: 'אוקטובר',
               11: 'נובמבר',
               12: 'דצמבר'}

In [ ]:
df['accident_month_hebrew'] = df['accident_month'].apply(lambda m: months_dict.get(m))

### Split to last 5 years and first 5 years

In [ ]:
first_5_years_start = pd.Timestamp(year=2018, month=7, day=1).date()

In [ ]:
df_first_5_years = df.loc[(df.accident_timestamp.dt.date < first_5_years_start)].copy()
df_last_5_years = df.loc[(df.accident_timestamp.dt.date >= first_5_years_start)].copy()

### validate 10 years

In [ ]:
total_number_of_years = 10

In [ ]:
assert(round(((df['accident_timestamp'].max() - df['accident_timestamp'].min()).days / 365),0) == total_number_of_years)


In [ ]:
timedelta = (df['accident_timestamp'].max() - df['accident_timestamp'].min())

In [ ]:
from math import isclose

In [ ]:
assert(isclose(timedelta.days / (365.25),10, abs_tol=1e-2))

In [ ]:
df['accident_timestamp'].max()

In [ ]:
df['accident_timestamp'].min()

### Create results dir

In [ ]:
schools_2023_results_directory = 'schools_2023_results'

In [ ]:
if not os.path.exists(schools_2023_results_directory):
    os.mkdir(schools_2023_results_directory)

In [ ]:
df.to_csv(os.path.join(schools_2023_results_directory,'all_injured_per_schools.csv'))

### create unique injured

In [ ]:
df_unique_injured = df.drop_duplicates('inv_unique_id').copy()

In [ ]:
df_unique_injured.to_csv(os.path.join(schools_2023_results_directory,'all_unique_injured.csv'))

In [ ]:
df_injured_in_all_schools = df.groupby(['injury_severity'])["inv_unique_id"].nunique()

In [ ]:
df_injured_in_all_schools.to_csv(os.path.join(schools_2023_results_directory,'df_injured_in_all_schools.csv'), index=False)

In [ ]:
df_injured_in_all_schools_per_v_or_p = df.groupby(['injury_severity', 'vehicle_or_pedestrian'])["inv_unique_id"].nunique()

In [ ]:
df_injured_in_all_schools_per_v_or_p.to_csv(os.path.join(schools_2023_results_directory,'df_injured_in_all_schools_per_type.csv'), index=False)

In [ ]:
df_injured_in_all_schools_per_p_c = df.groupby(['injury_severity', 'provider_code'])["inv_unique_id"].nunique()

In [ ]:
df_injured_in_all_schools_per_p_c.to_csv(os.path.join(schools_2023_results_directory,'df_injured_in_all_schools_per_p_c.csv'), index=False)                  

### create vehicle_or_pedestrian stats per 5 years (first 5 years, last 5 years)

In [ ]:
df_injured_in_all_schools_per_v_or_p_first_5_years = df_first_5_years.groupby(['injury_severity_hebrew', 'vehicle_or_pedestrian'])["inv_unique_id"].nunique().to_frame()
df_injured_in_all_schools_per_v_or_p_first_5_years.columns = ['count']
df_injured_in_all_schools_per_v_or_p_first_5_years.to_csv(os.path.join(schools_2023_results_directory,'df_injured_first_5_years_per_severity_vehicle_pedestrian.csv'), index=True)


In [ ]:
df_injured_in_all_schools_per_v_or_p_age_first_5_years = df_first_5_years.groupby(['injury_severity', 'injury_severity_hebrew', 'vehicle_or_pedestrian', 'age_group', 'age_group_hebrew'])["inv_unique_id"].nunique().to_frame()
df_injured_in_all_schools_per_v_or_p_age_first_5_years.columns = ['count']
df_injured_in_all_schools_per_v_or_p_age_first_5_years = df_injured_in_all_schools_per_v_or_p_age_first_5_years.reset_index().drop(columns=['injury_severity', 'age_group'])
df_injured_in_all_schools_per_v_or_p_age_first_5_years.to_csv(os.path.join(schools_2023_results_directory,'df_injured_first_5_years_per_severity_vehicle_pedestrian_age_group.csv'), index=False)


In [ ]:
df_injured_in_all_schools_per_v_or_p_last_5_years = df_last_5_years.groupby(['injury_severity_hebrew', 'vehicle_or_pedestrian'])["inv_unique_id"].nunique().to_frame()
df_injured_in_all_schools_per_v_or_p_last_5_years.columns = ['count']
df_injured_in_all_schools_per_v_or_p_last_5_years.to_csv(os.path.join(schools_2023_results_directory,'df_injured_last_5_years_per_severity_vehicle_pedestrian_age_group.csv'), index=True)


In [ ]:
df_injured_in_all_schools_per_v_or_p_age_last_5_years = df_last_5_years.groupby(['injury_severity', 'injury_severity_hebrew', 'vehicle_or_pedestrian', 'age_group', 'age_group_hebrew'])["inv_unique_id"].nunique().to_frame()
df_injured_in_all_schools_per_v_or_p_age_last_5_years.columns = ['count']
df_injured_in_all_schools_per_v_or_p_age_last_5_years = df_injured_in_all_schools_per_v_or_p_age_last_5_years.reset_index().drop(columns=['injury_severity', 'age_group'])
df_injured_in_all_schools_per_v_or_p_age_last_5_years.to_csv(os.path.join(schools_2023_results_directory,'df_injured_last_5_years_per_severity_vehicle_pedestrian_age_group.csv'), index=False)


In [ ]:
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years = df_last_5_years.groupby(['school_yishuv_name', 'injury_severity', 'injury_severity_hebrew', 'vehicle_or_pedestrian', 'age_group', 'age_group_hebrew'])["inv_unique_id"].nunique().to_frame()
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.columns = ['count']
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years = df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.reset_index().drop(columns=['injury_severity', 'age_group'])
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.sort_values(by=['school_yishuv_name', 'injury_severity_hebrew', 'vehicle_or_pedestrian', 'age_group_hebrew'], inplace=True)
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.to_csv(os.path.join(schools_2023_results_directory,'df_injured_last_5_years_per_severity_vehicle_pedestrian_age_group_yishuv.csv'), index=False)


In [ ]:
df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years = df_first_5_years.groupby(['school_yishuv_name', 'injury_severity', 'injury_severity_hebrew', 'vehicle_or_pedestrian', 'age_group', 'age_group_hebrew'])["inv_unique_id"].nunique().to_frame()
df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.columns = ['count']
df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years = df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.reset_index().drop(columns=['injury_severity', 'age_group'])
df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.sort_values(by=['school_yishuv_name', 'injury_severity_hebrew', 'vehicle_or_pedestrian', 'age_group_hebrew'], inplace=True)
df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.to_csv(os.path.join(schools_2023_results_directory,'df_injured_first_5_years_per_severity_vehicle_pedestrian_age_group_yishuv.csv'), index=False)


In [ ]:
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.loc[df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.vehicle_or_pedestrian.isin(['קורקינט חשמלי', 'אופניים חשמליים'])].groupby(['school_yishuv_name', 'injury_severity_hebrew'])['count'].sum().to_frame()



In [ ]:
tlv_jerusalem_p_last = df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.loc[df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.vehicle_or_pedestrian.isin(['הולך רגל'])&\
df_injured_in_all_schools_per_v_or_p_age_yishuv_last_5_years.school_yishuv_name.isin(['תל אביב - יפו', 'ירושלים'])].groupby(['school_yishuv_name', 'injury_severity_hebrew'])['count'].sum().to_frame().sort_values('count')
cols = tlv_jerusalem_p_last.columns
tlv_jerusalem_p_last.columns = [c for c in cols if c != 'count'] + ['count_pedestrians_last_5_years']
tlv_jerusalem_last

In [ ]:
tlv_jerusalem_p_first = df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.loc[df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.vehicle_or_pedestrian.isin(['הולך רגל'])&\
df_injured_in_all_schools_per_v_or_p_age_yishuv_first_5_years.school_yishuv_name.isin(['תל אביב - יפו', 'ירושלים'])].groupby(['school_yishuv_name', 'injury_severity_hebrew'])['count'].sum().to_frame().sort_values('count')
cols = tlv_jerusalem_p_first.columns
tlv_jerusalem_p_first.columns = [c for c in cols if c != 'count'] + ['count_pedestrians_first_5_years']
tlv_jerusalem_first

### prat weights

In [ ]:
killed_weight = 6600/7581
severe_weight = 956/7581
light_weight = 25/7581

### calc pedestrians

In [ ]:
df_only_pedestrians = df_unique_injured.loc[df_unique_injured.vehicle_or_pedestrian == 'הולך רגל'].copy()

In [ ]:
df_only_pedestrians.loc[df_only_pedestrians['cross_mode_hebrew'].isna(), 'cross_mode_hebrew'] = 'למס לא סיפקו נתונים'

In [ ]:
df_only_pedestrians.loc[df_only_pedestrians['didnt_cross_hebrew'].isna(), 'didnt_cross_hebrew'] = 'למס לא סיפקו נתונים'


In [ ]:
df_only_pedestrians.loc[df_only_pedestrians['cross_location_hebrew'].isna(), 'cross_location_hebrew'] = 'למס לא סיפקו נתונים'


In [ ]:
df_maavar = df_only_pedestrians.groupby(['injury_severity_hebrew','cross_location_hebrew']).size().to_frame()

In [ ]:
df_maavar.to_csv(os.path.join(schools_2023_results_directory,'df_maavar_pedestrians.csv'), index=True)

### calc per school

In [ ]:
df_total_injured_schools = (
    df_last_5_years.groupby(
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
            "injury_severity"
        ]
    )
    .size()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "injury_severity",
            "injured_count",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        ],
    ]
)
df_total_injured_schools = df_total_injured_schools.set_index(
    [
        "school_yishuv_name",
        "school_id",
        "school_name",
        "institution_type",
        "school_anyway_link",
        "school_longitude",
        "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        "injury_severity",

    ]
).unstack(-1)
df_total_injured_schools.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)

df_total_injured_schools.loc[:, (slice("injured_count"), slice(None))] = df_total_injured_schools.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured_schools["total_injured_count"] = (
    df_total_injured_schools.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

In [ ]:
df_total_injured_schools.columns = ['killed_count', 'severe_injured_count', 'light_injured_count', 'total_injured_count']


In [ ]:
df_total_injured_per_veh_or_ped = (
    df_last_5_years.groupby(
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
            "vehicle_or_pedestrian"
        ]
    )
    .size()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "vehicle_or_pedestrian",
            "injured_count",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        ],
    ]
)
df_total_injured_per_veh_or_ped = df_total_injured_per_veh_or_ped.set_index(
    [
        "school_yishuv_name",
        "school_id",
        "school_name",
        "institution_type",
        "school_anyway_link",
        "school_longitude",
        "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        "vehicle_or_pedestrian",

    ]
).unstack(-1)

df_total_injured_per_veh_or_ped.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)


In [ ]:
df_total_injured_per_veh_or_ped.columns = ['bike', 'electric_bike', 'pedestrians', 'electric_korki']

In [ ]:
df_total_accidents_schools = (
    df_last_5_years.groupby(
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        ]
    )["acc_unique_id"]
    .nunique()
    .reset_index(name="accidents_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
            "accidents_count"
        ],
    ]
)

df_total_accidents_schools = df_total_accidents_schools.set_index(
[
        "school_yishuv_name",
        "school_id",
        "school_name",
        "institution_type",
        "school_anyway_link",
        "school_longitude",
        "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6"])

In [ ]:
df_total = pd.merge(df_total_injured_schools,
                     df_total_accidents_schools,
                     left_index=True,
                     right_index=True,
                     how='left')

In [ ]:
df_total = pd.merge(df_total,
                     df_total_injured_per_veh_or_ped,
                     left_index=True,
                     right_index=True,
                     how='left')

In [ ]:
def calc_prat_score(x):
    prat_score = (x["killed_count"] + x["severe_injured_count"] + x["light_injured_count"]) * \
   (x["killed_count"] * killed_weight + \
    x["severe_injured_count"]  * severe_weight + \
    x["light_injured_count"]  * light_weight)
    return prat_score


In [ ]:
def calc_heuristic_score(x):
    h_score = x["killed_count"] * 8 + \
    x["severe_injured_count"]  * 5 + \
    x["light_injured_count"]  * 1
    return h_score

In [ ]:
df_total['prat_score'] = df_total.apply(lambda x: calc_prat_score(x), axis=1)

In [ ]:
df_total['h_score'] = df_total.apply(lambda x: calc_heuristic_score(x), axis=1)

In [ ]:
df_total = df_total.sort_values('prat_score', ascending=False)

In [ ]:
df_total.to_csv(os.path.join(schools_2023_results_directory,'scores_per_school_both_tik1_and_tik3_last_5_years.csv'))

### calc per yisuv

In [ ]:
df_total_injured = (
    df_last_5_years.groupby(
        [
            "school_yishuv_name",
            "injury_severity"
        ]
    )["inv_unique_id"]
    .nunique()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "injury_severity",
            "injured_count",
        ],
    ]
)
df_total_injured = df_total_injured.set_index(
    [
        "school_yishuv_name",
        "injury_severity",

    ]
).unstack(-1)
df_total_injured.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)


df_total_injured.loc[:, (slice("injured_count"), slice(None))] = df_total_injured.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured["total_injured_count"] = (
    df_total_injured.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

df_total_accidents_yishuvs = (
    df_last_5_years.groupby(
        [
            "school_yishuv_name",
        ]
    )["acc_unique_id"]
    .nunique()
    .reset_index(name="accidents_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "accidents_count"
        ],
    ]
)

df_total_accidents_yishuvs = df_total_accidents_yishuvs.set_index(
[
        "school_yishuv_name"])

df_total_yishuvs = pd.merge(df_total_injured,
                     df_total_accidents_yishuvs,
                     left_index=True,
                     right_index=True,
                     how='left')

df_total_yishuvs.columns = ['killed_count', 'severe_injured_count', 'light_injured_count', 'total_injured_count', 'accidents_count']

df_total_yishuvs['prat_score'] = df_total_yishuvs.apply(lambda x: calc_prat_score(x), axis=1)

df_total_yishuvs['h_score'] = df_total_yishuvs.apply(lambda x: calc_heuristic_score(x), axis=1)

df_total_yishuvs = df_total_yishuvs.sort_values('prat_score', ascending=False)

# calc per 'bike', 'electric_bike', 'pedestrians', 'electric_korki'

df_injured_per_yishuv_veh_or_ped = (
    df_last_5_years.groupby(
        [
            "school_yishuv_name",
            "vehicle_or_pedestrian"
        ]
    )["inv_unique_id"]
    .nunique()
    .reset_index(name="injured_count_per_type")
    .loc[
        :,
        [
            "school_yishuv_name",
            "injured_count_per_type",
            "vehicle_or_pedestrian",

        ],
    ])


df_injured_per_yishuv_veh_or_ped = df_injured_per_yishuv_veh_or_ped.set_index(
    [
            "school_yishuv_name",
            "vehicle_or_pedestrian",
    ]
).unstack(-1)

df_injured_per_yishuv_veh_or_ped.fillna({"injured_count_per_type": 0}, inplace=True)

df_injured_per_yishuv_veh_or_ped.columns = ['bike', 'electric_bike', 'pedestrians', 'electric_korki']


df_total_yishuvs = pd.merge(df_total_yishuvs,
                     df_injured_per_yishuv_veh_or_ped,
                     left_index=True,
                     right_index=True,
                     how='left')


In [ ]:
df_total_yishuvs.to_csv(os.path.join(schools_2023_results_directory,'scores_per_yishuv_both_tik1_and_tik3_last_5_years.csv'))

In [ ]:
top_20_yishuvs = df_total_yishuvs['prat_score'].sort_values(ascending=False).iloc[0:20]

In [ ]:
top_20_yishuvs = top_20_yishuvs.to_frame().reset_index()

### calc per yisuv and years

In [ ]:
curr_df = df_first_5_years.copy()

df_total_injured = (
    curr_df.groupby(
        [
            "school_yishuv_name",
            "injury_severity"
        ]
    )["inv_unique_id"]
    .nunique()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "injury_severity",
            "injured_count",
        ],
    ]
)
df_total_injured = df_total_injured.set_index(
    [
        "school_yishuv_name",
        "injury_severity",

    ]
).unstack(-1)
df_total_injured.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)


df_total_injured.loc[:, (slice("injured_count"), slice(None))] = df_total_injured.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured["total_injured_count"] = (
    df_total_injured.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

df_total_accidents_yishuvs = (
    curr_df.groupby(
        [
            "school_yishuv_name",
        ]
    )["acc_unique_id"]
    .nunique()
    .reset_index(name="accidents_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "accidents_count"
        ],
    ]
)

df_total_accidents_yishuvs = df_total_accidents_yishuvs.set_index(
[
        "school_yishuv_name"])

df_total_yishuvs_first_5_years = pd.merge(df_total_injured,
                     df_total_accidents_yishuvs,
                     left_index=True,
                     right_index=True,
                     how='left')

df_total_yishuvs_first_5_years.columns = ['killed_count', 'severe_injured_count', 'light_injured_count', 'total_injured_count', 'accidents_count']

df_total_yishuvs_first_5_years['prat_score'] = df_total_yishuvs_first_5_years.apply(lambda x: calc_prat_score(x), axis=1)

df_total_yishuvs_first_5_years['h_score'] = df_total_yishuvs_first_5_years.apply(lambda x: calc_heuristic_score(x), axis=1)

df_total_yishuvs_first_5_years = df_total_yishuvs_first_5_years.sort_values('prat_score', ascending=False)


In [ ]:

curr_df = df_last_5_years.copy()

df_total_injured = (
    curr_df.groupby(
        [
            "school_yishuv_name",
            "injury_severity"
        ]
    )["inv_unique_id"]
    .nunique()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "injury_severity",
            "injured_count",
        ],
    ]
)
df_total_injured = df_total_injured.set_index(
    [
        "school_yishuv_name",
        "injury_severity",

    ]
).unstack(-1)
df_total_injured.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)


df_total_injured.loc[:, (slice("injured_count"), slice(None))] = df_total_injured.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured["total_injured_count"] = (
    df_total_injured.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

df_total_accidents_yishuvs = (
    curr_df.groupby(
        [
            "school_yishuv_name",
        ]
    )["acc_unique_id"]
    .nunique()
    .reset_index(name="accidents_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "accidents_count"
        ],
    ]
)

df_total_accidents_yishuvs = df_total_accidents_yishuvs.set_index(
[
        "school_yishuv_name"])

df_total_yishuvs_last_5_years = pd.merge(df_total_injured,
                     df_total_accidents_yishuvs,
                     left_index=True,
                     right_index=True,
                     how='left')

df_total_yishuvs_last_5_years.columns = ['killed_count', 'severe_injured_count', 'light_injured_count', 'total_injured_count', 'accidents_count']

df_total_yishuvs_last_5_years['prat_score'] = df_total_yishuvs_last_5_years.apply(lambda x: calc_prat_score(x), axis=1)

df_total_yishuvs_last_5_years['h_score'] = df_total_yishuvs_last_5_years.apply(lambda x: calc_heuristic_score(x), axis=1)

df_total_yishuvs_last_5_years = df_total_yishuvs_last_5_years.sort_values('prat_score', ascending=False)



In [ ]:
df_total_yishuvs_first_5_years.columns = [c + '_13_18' for c in df_total_yishuvs_first_5_years.columns]

In [ ]:
df_total_yishuvs_last_5_years.columns = [c + '_18_23' for c in df_total_yishuvs_last_5_years.columns]

In [ ]:
df_all_yishuvs_per_5_years_group = pd.concat([df_total_yishuvs_first_5_years, df_total_yishuvs_last_5_years], axis=1)

In [ ]:
df_all_yishuvs_per_5_years_group['%change_prat'] = 100 * (df_all_yishuvs_per_5_years_group['prat_score_18_23'] - df_all_yishuvs_per_5_years_group['prat_score_13_18']) / df_all_yishuvs_per_5_years_group['prat_score_13_18']


In [ ]:
df_total_yishuvs_first_5_years.to_csv(os.path.join(schools_2023_results_directory,'scores_per_yishuv_first_5_years.csv'))


In [ ]:
df_total_yishuvs_last_5_years.to_csv(os.path.join(schools_2023_results_directory,'scores_per_yishuv_last_5_years.csv'))


In [ ]:
df_all_yishuvs_per_5_years_group.to_csv(os.path.join(schools_2023_results_directory,'scores_per_yishuv_5_years_periods.csv'))


In [ ]:
df_all_yishuvs_per_5_years_group_top_20_prat_18_23 = df_all_yishuvs_per_5_years_group.sort_values('prat_score_18_23', ascending=False)[0:20].sort_values('%change_prat', ascending=False)


In [ ]:
df_all_yishuvs_per_5_years_group_top_20_prat_18_23.to_csv(os.path.join(schools_2023_results_directory,'scores_per_yishuv_5_years_periods_top_20_prat_18_23_sorted_diff_prat.csv'))


### Choose Polygons

In [ ]:
def calc_dict_for_prat(df):
    killed = df.loc[df.loc[:,'injury_severity'] == 1].inv_unique_id.nunique()
    severe_injured = df.loc[df.loc[:,'injury_severity'] == 2].inv_unique_id.nunique()
    light_injured = df.loc[df.loc[:,'injury_severity'] == 3].inv_unique_id.nunique()
    total_accidents = df.acc_unique_id.nunique()
    
    final_dict = {"accidents_count": total_accidents,
                  "light_injured_count": light_injured,
                  "severe_injured_count": severe_injured,
                  "killed_count": killed}
    return final_dict


In [ ]:
MAX_TOP_SCHOOLS_IN_YISHUVS = 50
def get_final_poly_school_ids(yiushuv_name):
    print(yiushuv_name)
    df_yishuv_top_50_prat_schools = df_total[df_total.index.get_level_values('school_yishuv_name').isin([yiushuv_name])][0:MAX_TOP_SCHOOLS_IN_YISHUVS]
    final_combs_with_scores = []
    comb_size = df_yishuv_top_50_prat_schools.shape[0]
    if comb_size == 0:
        return []
    elif comb_size > 3:
        comb_size = 3
    else:
        pass
    all_school_ids = set(itertools.combinations(df_yishuv_top_50_prat_schools.index.values, comb_size))
    all_school_ids = [list(item) for item in set(tuple(row) for row in all_school_ids)]
    for comb in tqdm(all_school_ids):
        school_ids = [comb[0][1], comb[1][1], comb[2][1]]
        df_curr = df_last_5_years.loc[df_last_5_years.school_id.isin(school_ids)]
        prat_dict = calc_dict_for_prat(df_curr)
        prat_score = calc_prat_score(prat_dict)
        final_combs_with_scores.append({"school_ids": school_ids,
                                        "prat_score": prat_score})
    school_ids_final = pd.DataFrame(final_combs_with_scores).sort_values('prat_score', ascending=False)['school_ids'].iloc[0]
    return school_ids_final


In [ ]:
poly_school_ids_final = top_20_yishuvs.swifter.apply(lambda x: get_final_poly_school_ids(x['school_yishuv_name']), axis=1)


In [ ]:
poly_school_ids_final = poly_school_ids_final.sum()

### create final tables

In [ ]:
df_total_no_idx = df_total.reset_index()

In [ ]:
def get_bounding_box(latitude, longitude, distance_in_km):
    latitude = math.radians(latitude)
    longitude = math.radians(longitude)

    radius = 6371
    # Radius of the parallel at given latitude
    parallel_radius = radius * math.cos(latitude)

    lat_min = latitude - distance_in_km / radius
    lat_max = latitude + distance_in_km / radius
    lon_min = longitude - distance_in_km / parallel_radius
    lon_max = longitude + distance_in_km / parallel_radius
    rad2deg = math.degrees

    return rad2deg(lat_min), rad2deg(lon_min), rad2deg(lat_max), rad2deg(lon_max)

In [ ]:
all_relevant_schools = []
relevant_schools_per_polygon_mikbatz = {}
for school_id in poly_school_ids_final:
    center_lat, center_lon = df_total_no_idx.loc[df_total_no_idx.school_id == school_id, ['school_latitude', 'school_longitude']].values[0]
    lat_min, lon_min, lat_max, lon_max = get_bounding_box(center_lat, center_lon, 0.5)
    baseX = lon_min
    baseY = lat_min
    distanceX = lon_max
    distanceY = lat_max

    poly = Polygon([(baseX, baseY), 
                    (baseX, distanceY),
                    (distanceX, distanceY),
                    (distanceX, baseY)])
    bnbr_schools = df_total_no_idx.drop_duplicates(['school_id','school_longitude', 'school_latitude']).loc[:,['school_id','school_longitude', 'school_latitude']].to_dict(orient='records')
    schools_in_1km_box = [r['school_id'] for r in bnbr_schools if poly.contains(Point(r['school_longitude'], r['school_latitude']))]
    all_relevant_schools += schools_in_1km_box
    relevant_schools_per_polygon_mikbatz[school_id] = schools_in_1km_box


### polygons of final schools

In [ ]:
final_polygons_csv = df_total_no_idx.loc[df_total_no_idx.school_id.isin(poly_school_ids_final), ['WKT Polygon',
                                                                                                 'school_id',
                                                                                                'school_yishuv_name',
                                                                                                'school_name',
                                                                                                'prat_score',
                                                                                                'killed_count',
                                                                                                'severe_injured_count',
                                                                                                'light_injured_count',
                                                                                                'total_injured_count',
                                                                                                'pedestrians',
                                                                                                'bike',
                                                                                                'electric_bike',
                                                                                                'electric_korki',
                                                                                                'accidents_count']].reset_index(drop=True)

In [ ]:
final_polygons_csv['WKT'] = final_polygons_csv['WKT Polygon'].apply(lambda x: x.replace(",", ", "))

In [ ]:
final_polygons_csv['mikbatz_schools_ids'] = final_polygons_csv.school_id.apply(lambda school_id: relevant_schools_per_polygon_mikbatz[school_id])
final_polygons_csv['mikbatz_schools_names'] = final_polygons_csv['mikbatz_schools_ids'].apply(lambda schools_list: [df_total_no_idx.loc[df_total_no_idx.school_id == school_id, 'school_name'].iloc[0] for school_id in schools_list])
final_polygons_csv['mikbatz_schools_count'] = final_polygons_csv['mikbatz_schools_ids'].apply(lambda x: len(x))


In [ ]:
relevant_cols = ['WKT',
                 'school_id',
                 'school_name',
                 'school_yishuv_name',
                 'mikbatz_schools_ids',
                 'mikbatz_schools_names',
                 'mikbatz_schools_count',
                 'prat_score',
                 'killed_count',
                 'severe_injured_count',
                 'light_injured_count',
                 'total_injured_count',
                 'pedestrians',
                 'bike',
                 'electric_bike',
                 'electric_korki',
                 'accidents_count']

In [ ]:
final_polygons_csv = final_polygons_csv.loc[:, relevant_cols]

In [ ]:
final_polygons_csv['school_name'] = 'מקבץ ' + final_polygons_csv['school_name']

In [ ]:
final_polygons_csv['prat_score'] = final_polygons_csv['prat_score'].round(0).astype(int)

In [ ]:
relevant_hebrew_cols = ['WKT',
                        'id של בית ספר',
                  'שם מקבץ',
                  'ישוב',
                  'ids של בתי ספר במקבץ',
                  'שמות בתי ספר במקבץ',
                  'ספירה של בתי ספר במקבץ',
                  'דירוג',
                  'סה״כ הרוגים במקבץ',
                  'סה״כ פצועים קשה במקבץ',
                  'סה״כ פצועים קל במקבץ',
                  'סה״כ נפגעים והרוגים',
                  'סה״כ הולכי רגל נפגעים',
                  'סה״כ נפגעים רוכבי אופניים',
                  'סה״כ נפגעים רוכבי אופניים חשמלים',
                  'סה״כ נפגעים רוכבי קורקינט חשמלי',
                  'מספר תאונות']

In [ ]:
final_polygons_csv.columns = relevant_hebrew_cols

In [ ]:
final_polygons_csv.to_csv(os.path.join(schools_2023_results_directory, 'final_polygons_csv.csv'), index=False)

### final schools and injured csvs for maps

In [ ]:
final_schools_csv = df_total_no_idx.loc[df_total_no_idx.school_id.isin(all_relevant_schools), ['school_yishuv_name', 'school_name', 'school_longitude', 'school_latitude']]


In [ ]:
final_schools_csv.columns = ['ישוב' , 'שם מוסד', 'longitude', 'latitude']

In [ ]:
final_schools_csv.to_csv(os.path.join(schools_2023_results_directory, 'final_schools_csv.csv'), index=False)


In [ ]:
final_injured_csv = df_last_5_years.loc[df_last_5_years.school_id.isin(all_relevant_schools), ['vehicle_or_pedestrian',  'injury_severity_hebrew', 'age_group_hebrew', 'accident_timestamp' ,'longitude', 'latitude', 'inv_unique_id']].drop_duplicates('inv_unique_id')


In [ ]:
final_injured_csv = final_injured_csv.loc[:, ['vehicle_or_pedestrian', 'injury_severity_hebrew' ,'age_group_hebrew', 'accident_timestamp', 'longitude', 'latitude']].reset_index(drop=True)


In [ ]:
final_injured_csv.columns = ['סוג נפגע', 'חומרת פגיעה', 'קבוצת גיל' ,'תאריך ושעה' ,'longitude', 'latitude']

### remove injured not in bb

In [ ]:
injured_idx_in_poly = set()
for school_id in tqdm(poly_school_ids_final):
    center_lat, center_lon = df_total_no_idx.loc[df_total_no_idx.school_id == school_id, ['school_latitude', 'school_longitude']].values[0]
    lat_min, lon_min, lat_max, lon_max = get_bounding_box(center_lat, center_lon, 0.5)
    baseX = lon_min
    baseY = lat_min
    distanceX = lon_max
    distanceY = lat_max

    poly = Polygon([(baseX, baseY), 
                    (baseX, distanceY),
                    (distanceX, distanceY),
                    (distanceX, baseY)])
    injured = final_injured_csv.loc[:,['longitude', 'latitude']].to_dict(orient='index')
    injured_idx = [k for k, v in injured.items() if poly.contains(Point(v['longitude'], v['latitude']))]
    injured_idx_in_poly = injured_idx_in_poly.union(set(injured_idx))
    

In [ ]:
final_injured_csv = final_injured_csv.loc[list(injured_idx_in_poly)]

In [ ]:
final_injured_csv.to_csv(os.path.join(schools_2023_results_directory, 'final_injured_csv.csv'), index=False)

In [ ]:
final_injured_csv_light = final_injured_csv.loc[final_injured_csv['חומרת פגיעה'] == 'פצוע קל']

final_injured_csv_light.to_csv(os.path.join(schools_2023_results_directory, 'final_injured_csv_light.csv'), index=False)

In [ ]:
final_injured_csv_severe = final_injured_csv.loc[final_injured_csv['חומרת פגיעה'] == 'פצוע קשה']

final_injured_csv_severe.to_csv(os.path.join(schools_2023_results_directory, 'final_injured_csv_severe.csv'), index=False)

In [ ]:
final_injured_csv_killed = final_injured_csv.loc[final_injured_csv['חומרת פגיעה'] == 'הרוג']

final_injured_csv_killed.to_csv(os.path.join(schools_2023_results_directory, 'final_injured_csv_killed.csv'), index=False)


### create jsons

#### get all schools

In [ ]:
schools_data = json.load(open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/schools_names.json"))


In [ ]:
all_schools_df = pd.DataFrame(schools_data)

In [ ]:
all_schools_ids = all_schools_df.school_id.unique()

#### injured_around_schools_api_2023_json

In [ ]:
curr_df = df_last_5_years.copy()

df_total_injured_schools_per_year = (
    curr_df.groupby(
        [
            "school_id",
            "accident_year",   
            "injury_severity"
        ]
    )
    .size()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_id",
            "accident_year",
            "injury_severity",
            "injured_count",
        ],
    ]
)
df_total_injured_schools_per_year = df_total_injured_schools_per_year.set_index(
    [
            "school_id",
            "accident_year",   
            "injury_severity"

    ]
).unstack(-1)
df_total_injured_schools_per_year.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)

df_total_injured_schools_per_year.loc[:, (slice("injured_count"), slice(None))] = df_total_injured_schools_per_year.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured_schools_per_year["total_injured_count"] = (
    df_total_injured_schools_per_year.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

df_total_injured_schools_per_year.columns = ['killed_count', 'severly_injured_count', 'light_injured_count', 'total_injured_killed_count']


df_total_injured_schools_per_year = df_total_injured_schools_per_year.reset_index(drop=False).set_index('school_id')

injured_schools_per_year_list = df_total_injured_schools_per_year.reset_index().to_dict(orient='records')

school_ids = df_total_injured_schools_per_year.index.unique()

total_dictionary_for_json_api = {}
for school_id in all_schools_ids:
    total_dictionary_for_json_api[str(school_id)] = [r for r in injured_schools_per_year_list if r["school_id"] == school_id]
with open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/injured_around_schools_api_2023.json", "w") as f:
    json.dump(total_dictionary_for_json_api, f)


#### injured_around_months_graph_data_api_2023_json

In [ ]:
df_total_injured_schools_per_month = (
    curr_df.groupby(
        [
            "school_id",
            "accident_month_hebrew",   
        ]
    )
    .size()
    .reset_index(name="count_1")
    .loc[
        :,
        [
            "school_id",
            "accident_month_hebrew",
            "count_1",
        ],
    ]
)
injured_schools_per_month_list = df_total_injured_schools_per_month.to_dict(orient='records')
total_dictionary_for_json_api = {}
for school_id in all_schools_ids:
    total_dictionary_for_json_api[str(school_id)] = [r for r in injured_schools_per_month_list if r["school_id"] == school_id]
with open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/injured_around_schools_months_graphs_data_api_2023.json", "w") as f:
    json.dump(total_dictionary_for_json_api, f)


#### injured_around_sex_graph_data_api_2023_json


In [ ]:
df_total_injured_schools_per_sex = (
    curr_df.groupby(
        [
            "school_id",
            "sex_hebrew",   
        ]
    )
    .size()
    .reset_index(name="count_1")
    .loc[
        :,
        [
            "school_id",
            "sex_hebrew",
            "count_1",
        ],
    ]
)
injured_schools_per_sex_list = df_total_injured_schools_per_sex.to_dict(orient='records')
total_dictionary_for_json_api = {}
for school_id in all_schools_ids:
    total_dictionary_for_json_api[str(school_id)] = [r for r in injured_schools_per_sex_list if r["school_id"] == school_id]
with open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/injured_around_schools_sex_graphs_data_api_2023.json", "w") as f:
    json.dump(total_dictionary_for_json_api, f)


### calculate ncr - reason for choosing only 50 as MAX_TOP_SCHOOLS_IN_YISHUVS

In [ ]:
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom  # or / in Python 2

In [ ]:
ncr(50,3)

In [ ]:
ncr(100,3)

In [ ]:
ncr(193,3)